In [1]:
import cv2
import numpy as np
import serial
import time
import tkinter as tk
from PIL import Image, ImageTk
from collections import deque

# === SERIAL SETUP ===
try:
    esp = serial.Serial('COM3', 115200, timeout=1)
    time.sleep(2)
    print("✅ Connected to ESP32 on COM3")
except:
    esp = None
    print("❌ Could not connect to ESP32 on COM3")

# === HSV Range for BLUE color ===
lower_hsv = np.array([100, 150, 0])
upper_hsv = np.array([140, 255, 255])

# === EXTERNAL CAMERA (usually index 1) ===
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("❌ External camera not found on index 1.")
    exit()
else:
    print("✅ External camera detected.")

# === TKINTER GUI ===
root = tk.Tk()
root.title("External Camera - Color Tracker")

panel = tk.Label(root)
panel.pack()

positions = deque(maxlen=5)  # For smoothing

def update_frame():
    ret, frame = cap.read()
    if not ret:
        root.after(10, update_frame)
        return

    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    h, w, _ = frame.shape
    center_x = w // 2
    deadzone = 40

    cv2.line(frame, (center_x, 0), (center_x, h), (0, 0, 255), 2)

    if contours:
        c = max(contours, key=cv2.contourArea)
        if cv2.contourArea(c) > 800:
            x, y, cw, ch = cv2.boundingRect(c)
            obj_x = x + cw // 2
            positions.append(obj_x)
            avg_x = int(np.mean(positions))

            # Draw visuals
            cv2.rectangle(frame, (x, y), (x+cw, y+ch), (0, 255, 0), 2)
            cv2.circle(frame, (avg_x, y + ch // 2), 5, (255, 0, 0), -1)

            # Send command to ESP32
            if esp:
                if avg_x < center_x - deadzone:
                    esp.write(b"L")
                elif avg_x > center_x + deadzone:
                    esp.write(b"R")
                else:
                    esp.write(b"C")

    # Display in Tkinter
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(image=img)

    panel.imgtk = imgtk
    panel.config(image=imgtk)

    root.after(30, update_frame)

def on_close():
    if cap.isOpened():
        cap.release()
    if esp:
        esp.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_close)
update_frame()
root.mainloop()


✅ Connected to ESP32 on COM3
✅ External camera detected.


In [7]:
import matplotlib.pyplot as plt


In [5]:
pip install pyserial


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
pip install numpy matplotlib


   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.8 MB 3.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/12.8 MB 3.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/12.8 MB 2.9 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.8 MB 2.4 MB/s eta 0:00:05
   --------- ------------------------------ 2.9/12.8 MB 2.3 MB/s eta 0:00:05
   --------- ------------------------------ 3.1/12.8 MB 2.2 MB/s eta 0:00:05
   ------------ --------------------------- 3.9/12.8 MB 2.3 MB/s eta 0:00:04
   ---------------- ----------------------- 5.2/12.8 MB 2.8 MB/s eta 0:00:03
   ------------------- -------------------- 6.3/12.8 MB 3.0 MB/s eta 0:00:03
   ---------------------- ----------------- 7.3/12.8 MB 3.2 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.8 MB 3.2 MB/s eta 0:00:02
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
